## Setting up the environment


In [22]:
import gymnasium as gym
import text_flappy_bird_gym

if __name__ == "__main__":
    env = gym.make("TextFlappyBird-v0", height=15, width=20, pipe_gap=4)

# Implement Q method

In [23]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [24]:
class Q_Learning:
    def __init__(self, space_size, action_size, gamma=1):
        """
        Calculates optimal policy using in-policy Temporal Difference control
        Evaluates Q-value for (S,A) pairs, using one-step updates.
        """
        # the discount factor
        self.gamma = gamma

        # size of system
        self.space_size = space_size  # as tuple
        self.action_size = action_size

        # where to save returns
        self.Qvalues = np.zeros((*self.space_size, self.action_size))
    
    def get_action_epsilon_greedy(self, s):
        """
        Chooses action at random using an epsilon-greedy policy wrt the current Q(s,a).
        """
        ran = np.random.rand()

        if ran < self.epsilon:
            prob_actions = np.ones(self.action_size) / self.action_size

        else:
            best_value = np.max(self.Qvalues[(*s,)])

            best_actions = self.Qvalues[(*s,)] == best_value

            prob_actions = best_actions / np.sum(best_actions)

        a = np.random.choice(self.action_size, p=prob_actions)
        return a
    
    def single_step_update(self, s, a, r, new_s, done):
        """
        Uses a single step to update the values, using Temporal Difference for Q values.
        Unlike sarsa, the new action new_a is not used, as it is chosen as the argmax of Q wrt s_new.
        In this case it is stored as it is needed for the implementation of Q(lambda)
        """
        if self.old_action is not None and a != self.old_action:
            self.et = np.zeros((*self.space_size, self.action_size))
        else:
            self.et *= self.gamma*self.lambda_  

        self.et[(*s, a)] += 1

        if done:
            deltaQ = r + 0 - self.Qvalues[(*s, a)]
            self.Qvalues += self.lr_v*deltaQ*self.et

        else:
            maxQ_over_actions = np.max(self.Qvalues[(*new_s,)])
            a_star = np.argmax(self.Qvalues[(*new_s,)])
            self.old_action = a_star

            deltaQ = r + self.gamma * maxQ_over_actions - self.Qvalues[(*s, a)]

            self.Qvalues += self.lr_v*deltaQ*self.et
    
    def train(self, n_episodes = 10000, lambda_ = 0, tstar = None, epsilon_0=0.2, k_epsilon = 0, lr_v0=0.15, k_lr = 0):
        """
        This function trains the agent using n_episodes.
        The default parameters use constant learning rate and epsilon (k = 0 in both cases)
        Otherwise a decaying rate is implemented after a starting point t0 (see README for more details)
        Similarly the default implements a TD(0) evaluation procedure (lambda = 0)
        """

        Actions = [i for i in range(self.action_size)]
        self.n_episodes = n_episodes

        # Add the following attributes to the class
        self.performance_traj = np.zeros(n_episodes) # To store cumulative reward at every game
        
        self.et = np.zeros((*self.space_size, self.action_size))
        self.lambda_ = lambda_

        # Parameters for epsilon decay
        self.epsilon_0 = epsilon_0  # Needed to name the plots
        self.epsilon = epsilon_0    # Needed to keep track of current epsilon
        self.k_epsilon = k_epsilon

        # Parameters for learning rate decay
        self.lr_v0 = lr_v0
        self.lr_v = lr_v0
        self.k_lr = k_lr

        self.old_action = None # Needed for Q(lamba) learning

        if(tstar is None):
            tstar = 2.5 * n_episodes 
        
        count = 0 # counter variable needed to see when to start decaying rates

        # Run over episodes
        for i in range(n_episodes):
            done = False
            s, info = env.reset()
            a = self.get_action_epsilon_greedy(s)
            act = Actions[a]

            while not done:
                count += 1

                # Perform one "step" in the environment
                new_s, r, done, _, info = env.step(act)

                # Keep track of rewards for one episode
                self.performance_traj[i] += r

                # Determine what action the agent would choose according to optimal policy
                new_a = self.get_action_epsilon_greedy(new_s)
                act = Actions[new_a]
                # Single update with (S, A, R', S')
                self.single_step_update(s, a, r, new_s, done)

                if count > tstar:
                    self.epsilon = epsilon_0/(1. + self.k_epsilon*(count - tstar)**1.05)
                    self.lr_v = lr_v0/(1 + self.k_lr*(count - tstar)**0.75)
                
                a = new_a
                s = new_s
    
    def plot_traj(self, cumulative=True, local=False, save = False):

        title = "Q Learning"

        plot_indexes = np.arange(0, self.n_episodes + 1, 20, dtype=int)
        plot_indexes[-1] = plot_indexes[-1] - 1

        plt.plot(plot_indexes, self.performance_traj[plot_indexes])

        if cumulative:
            cumulative_mean = np.cumsum(self.performance_traj) / np.arange(1, len(self.performance_traj) + 1)
            plt.plot(plot_indexes, cumulative_mean[plot_indexes], label="Cumulative mean")
        
        if local:
            window_size = 100
            local_mean_SARSA = np.convolve(self.performance_traj, np.ones(window_size) / window_size, mode="valid")
            plt.plot(plot_indexes[plot_indexes < local_mean_SARSA.shape[0]],
                     local_mean_SARSA[plot_indexes[plot_indexes < local_mean_SARSA.shape[0]]],
                     label=" Local Mean", color = 'red')
            
        plt.xlabel("Episode")
        plt.ylabel("Episode reward")
        plt.legend()
        plt.suptitle(f"{title} control cumulative rewards")

        plt.title(f"$\epsilon_0$ = {self.epsilon_0}, $k_\epsilon$ = {self.k_epsilon}, $\\alpha_0$ = {self.lr_v0}, $k_{{\\alpha}}$ = {self.k_lr}, $\lambda$ = {self.lambda_}")
        plt.tight_layout()
        
        if save:
            name = (
                "Plots/Q_plots/Q_lambda" + str(self.lambda_)
                + "_k_alpha_"
                + str(self.k_lr)
                + "k_epsilon"
                + str(self.k_epsilon)
                + ".png"
            )
            plt.savefig(name)
        plt.show()

In [28]:
observation_space = (env.observation_space[0].n, env.observation_space[1].n)
n_episodes = 10000

In [29]:
lambdas = [0.0, 0.2, 0.5, 0.8]
k_epsilons = [0.00005, 0.0005, 0.005, 0.05]
k_lrs = [0.00003, 0.0003, 0.003, 0.03]

In [ ]:
for i in lambdas:
    for j in k_epsilons:
        for l in k_lrs:
            q_learning = Q_Learning(space_size=observation_space, action_size=2, gamma=1)
            q_learning.train(n_episodes=n_episodes, lambda_=i, k_epsilon=j, k_lr=l)
            q_learning.plot_traj(cumulative=True, local=True, save=True)